# Proof of Concept Testing - Pre-trained Tacotron2

This notebook is intended to show proof of concept for using Coqui-AI to convert ebooks into audiobooks. In order to accomplish this, the notebook will use tools from the epud-parser notebook, as well as the Coqui-AI / TTS engine tools to convert the parsed text into audio files.

**Note:** this notebook is intended for use with the `ttsenv` conda environment.

## Outline of steps

1. Convert .epub file to text string
2. Run text through TTS engine
3. Combine audio files

In [1]:
import ebooklib
import torch
from ebooklib import epub
from bs4 import BeautifulSoup

In [2]:
# ?torch.hub.load

In [4]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# map_location=torch.device('cpu')
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', map_location=torch.device('cpu'), model_math='fp16')
# tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

Using cache found in /home/matthew/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

## Step 1 - Epub to text

Text here.

In [ ]:
book = epub.read_epub('pg2554.epub')
# book = epub.read_epub('epub-test.epub')

Text here.

In [ ]:
# for item in book.get_items():
#    if item.get_type() == ebooklib.ITEM_DOCUMENT:
#        print('==================================')
#        print('NAME : ', item.get_name())
#        print('----------------------------------')
#        print(BeautifulSoup(item.get_content(), "html.parser").text)
#        print('==================================')

## Step 2 - Tacotron2 from TorchHub
Text here.

In [ ]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

In [ ]:
text = "Hello world, I missed you so much."

In [ ]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence([text])

In [ ]:
with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22050

In [ ]:
from scipy.io.wavfile import write
write("audio.wav", rate, audio_numpy)

In [ ]:
from IPython.display import Audio
Audio(audio_numpy, rate=rate)

In [ ]:
# i = 0
# for item in book.get_items():
#     if item.get_type() == ebooklib.ITEM_DOCUMENT:
#         i += 1
#         if i == 2:
#             # set up text sample and path
#             input_text = BeautifulSoup(item.get_content(), "html.parser").text
#             sample_path = "output"+str(i)+".wav"

#             print(input_text)
            
#             # run text sample through TTS engine
#             !tts --text "$input_text" --model_name "tts_models/en/ek1/tacotron2" --out_path $sample_path
        